In [4]:
cd /home/chenxupeng/projects/deepshape/

/home/chenxupeng/projects/deepshape


In [6]:
from keras.models import Sequential, Model
from keras.layers import *
from keras.layers.advanced_activations import LeakyReLU
from keras.activations import relu
from keras.initializers import RandomNormal
from keras.applications import *
import keras.backend as K
from tensorflow.contrib.distributions import Beta
import tensorflow as tf
from keras.optimizers import Adam
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import argparse, sys, os, errno
from glob import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import h5py
from tqdm import tqdm
from keras.callbacks import TensorBoard
from keras.callbacks import EarlyStopping
from unet_128_model_row_column import *
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
%pylab inline
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" # so the IDs match nvidia-smi
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.99
set_session(tf.Session(config=config))

Populating the interactive namespace from numpy and matplotlib


In [25]:
def __conv_init(a):
    print("conv_init", a)
    k = RandomNormal(0, 0.02)(a) # for convolution kernel
    k.conv_weight = True    
    return k
conv_init = RandomNormal(0, 0.02)
def mixed_scaled_dense_network(input_tensor, num_layers=32): #, nc_in=3, nc_out=1
    """
    Inefficient implementation of paper: A mixed-scale dense convolutional neural network for image analysis
    http://www.pnas.org/content/115/2/254
    """
    msd_layers = {}
    x = input_tensor
    msd_layers["input"] = x
    for i in range(num_layers):
        dilation = (i % 10) + 1
        msd_layers["layer{0}".format(i)] = Conv2D(1, kernel_size=3, strides=1, dilation_rate=dilation, 
                                                  kernel_initializer=conv_init, padding="same")(x)
        for j in range(i):
            dilation = ((i + j) %10) + 1
            conv_3x3 = Conv2D(1, kernel_size=3, strides=1, dilation_rate=dilation,
                              kernel_initializer=conv_init, use_bias=True, padding="same")(msd_layers["layer{0}".format(j)])
            msd_layers["layer{0}".format(i)] = add([msd_layers["layer{0}".format(i)], conv_3x3])
        msd_layers["layer{0}".format(i)] = Activation("relu")(msd_layers["layer{0}".format(i)])
    
    concat_all = x
    for i in range(num_layers):
        concat_all = concatenate([concat_all, msd_layers["layer{0}".format(i)]])
    msd_layers["merge_concat_all"] = concat_all
    out = Conv2D(nc_out, kernel_size=1, kernel_initializer=conv_init, padding="same")(concat_all)
    msd_layers["output"] = out
    
    return out, msd_layers

In [26]:
number = 100
with h5py.File('/home/chenxupeng/projects/deepshape/data/new/train_0') as f:
    images_train = f['train_images'][:number]
#with h5py.File('/home/chenxupeng/projects/deepshape/data/new/test_0') as f:
    #images_test= f['test_images'][:number]
with h5py.File('/home/chenxupeng/projects/deepshape/data/new/Spitale_2015_invivo_CDS_0.1') as f:
    y_train = f['y_train'][:number]
    #y_test = f['y_test'][:number]
y_train = np.concatenate([y_train,y_train],axis = 1)
images_train.shape

(100, 128, 128, 16)

In [28]:
mixed_scaled_dense_network(images_train,32)

ValueError: Layer conv2d_5 was called with an input that isn't a symbolic tensor. Received type: <type 'numpy.ndarray'>. Full input: [array([[[[ 0.,  0.,  0., ...,  0.,  0.,  1.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         ..., 
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

        [[ 0.,  0.,  0., ...,  0.,  1.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         ..., 
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

        [[ 0.,  0.,  0., ...,  1.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         ..., 
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0., 